In [ ]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import urbanpy as up
import pandas as pd
import geopandas as gpd
import contextily as cx
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import time
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from matplotlib.lines import Line2D
from tqdm.auto import tqdm

In [ ]:
tqdm.pandas() # Activate progress bar for pandas apply

In [ ]:
def accessibility_analysis(hexagons, pois):
    # Calculate the Nearest Facility for each Hexagon
    hexagons['lon'] = hexagons.geometry.centroid.x
    hexagons['lat'] = hexagons.geometry.centroid.y
    
    dists, ixs = up.utils.nn_search(
        tree_features=pois[['lat', 'lon']].values,
        query_features=hexagons[['lat', 'lon']].values,
        metric='haversine'
    )
    
    hexagons["nearest_poi_ix"] = ixs
    
    # Calculate travel times and distances
    distance_duration = hexagons.progress_apply(
        lambda row: up.routing.osrm_route(
            origin=row.geometry.centroid, 
            destination = pois.iloc[row['nearest_poi_ix']]['geometry']
        ),
        result_type='expand',
        axis=1,
    )
    
    # Add columns to dataframe
    hexagons['distance_to_nearest_poi'] =  distance_duration[0] / 1000 # meters to km
    hexagons['duration_to_nearest_poi'] = distance_duration[1] / 60 # seconds to minutes
    
    custom_bins, custom_labels = up.utils.create_duration_labels(hexagons['duration_to_nearest_poi'])
    hexagons['duration_to_nearest_poi_label'] = pd.cut(hexagons['duration_to_nearest_poi'], bins=custom_bins, labels=custom_labels)
        
    return hexagons

In [ ]:
def gen_map(city_str, pop_df, pop_col, hex_res, poi_type, osrm_server, city_id=0):
    city = up.download.nominatim_osm(city_str, city_id)
    city.plot()
    plt.show()
    
    hexs_city = up.geom.gen_hexagons(resolution=hex_res, city=city)
    print("hex num:", hexs_city.shape[0])
    
    city_pop = up.geom.filter_population(pop_df, city)
    hexs_city_pop = up.geom.merge_shape_hex(hexs_city, city_pop, agg={pop_col: "sum"})
    
    city_hf = up.download.overpass_pois(city.total_bounds, poi_type)
    print("poi num:", city_hf.shape[0])
    
    up.routing.start_osrm_server(osrm_server["country"], osrm_server["continent"], "foot")
    time.sleep(20) # Big server
    
    for i in range(5):
        try:
            hexs_city_pop_access = accessibility_analysis(hexagons=hexs_city_pop, pois=city_hf)
            
            up.routing.stop_osrm_server(osrm_server["country"], osrm_server["continent"], "foot")
            
            fig, ax = plt.subplots(1, 1, figsize=(10, 10))
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="5%", pad=0.1)
            hexs_city_pop_access.query(f"{pop_col} > 0").plot(f"{pop_col}", alpha=0.5, legend=True, cax=cax, ax=ax)
            ax.set_title(f'Population \nat H3 hexagons resolution {hex_res}')
            ax.set_axis_off()
            cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, crs='EPSG:4326')
            # plt.savefig('outputs/static_maps/population_brasilia.png', dpi=300, bbox_inches='tight')
            plt.show()

            fig, ax = plt.subplots(1, 1, figsize=(10, 10))
            hexs_city_pop_access.query(f"{pop_col} > 0").plot("duration_to_nearest_poi_label", cmap='magma_r',
                                                              alpha=0.5, legend=True, ax=ax)
            ax.set_title(f'Accessibility to Health Facilities\nat H3 hexagons resolution {hex_res}')
            ax.set_axis_off()
            cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik, crs='EPSG:4326')
            plt.savefig(f"{city_str.lower().replace(' ', '_')}.png", dpi=300, bbox_inches='tight')
            plt.show()
            
            break
        except Exception as e:
            print("Exception:", e)
            time.sleep(5)
    
    return hexs_city_pop_access

In [ ]:
def interactive_map(gdf, city_str, pop_col, labels):
    fig = up.plotting.choropleth_map(
        title=f'Access to Health Places in {city_str}',
        gdf=gdf.query(f'{pop_col} > 0').reset_index(drop=True), 
        color_column='duration_to_nearest_poi_label',
        color_discrete_sequence=px.colors.sequential.Magma_r, 
        category_orders={'duration_to_nearest_poi_label': labels}, 
        labels={'duration_to_nearest_poi_label': 'Minutes'},
        zoom=9, opacity=0.5,
    )

    fig.update_layout(
        margin=dict(l=0, r=0, b=0),
        mapbox_style='open-street-map',
    )

    fig.update_traces(marker_line_width=0)
    fig.write_html(f"{city_str.lower().replace(' ', '_')}.html")
    fig.show()
    

In [ ]:
# Download country population
brasil_pop = up.download.hdx_fb_population(country="brazil", map_type="elderly")

In [ ]:
# Population parameters
pop_df=brasil_pop
pop_col="population"

In [ ]:
# General parameters
hex_res=7
poi_type='health'
osrm_server={"country":"sudeste", "continent":"south-america_brazil"}

In [ ]:
city_str='São Paulo, Brazil'
hexs_sp_pop_access = gen_map(city_str, pop_df, pop_col, hex_res, poi_type, osrm_server)

In [ ]:
city_str='Rio de Janeiro, Brazil'
hexs_rj_pop_access = gen_map(city_str, pop_df, pop_col, hex_res, poi_type, osrm_server)

In [ ]:
city_str='Campinas, Brazil'
hexs_campinas_pop_access = gen_map(city_str, pop_df, pop_col, hex_res, poi_type, osrm_server)

In [ ]:
# Specific parameters for Brasilia
city_str="Brasília, Distrito Federal, Brazil"
city_id=1
hex_res=8
osrm_server={"country":"centro-oeste", "continent":"south-america_brazil"}

In [ ]:
hexs_brasilia_pop_access = gen_map(city_str, pop_df, pop_col, hex_res, poi_type, osrm_server, city_id=city_id)

In [ ]:
# Specific parameters for Florianopolis
city_str="Florianopolis"
hex_res=8
poi_type='education'
osrm_server={"country":"sul", "continent":"south-america_brazil"}

In [ ]:
hexs_florianopolis_pop_access = gen_map(city_str, pop_df, pop_col, hex_res, poi_type, osrm_server)

In [ ]:
plotly.offline.init_notebook_mode()

In [ ]:
labels = hexs_sp_pop_access['duration_to_nearest_poi_label'].unique().sort_values()

In [ ]:
interactive_map(hexs_rj_pop_access, 'Rio de Janeiro', pop_col, labels)

In [ ]:
interactive_map(hexs_sp_pop_access, 'Sao Paulo', pop_col, labels)

In [ ]:
interactive_map(hexs_campinas_pop_access, 'Campiñas', pop_col, labels)

In [ ]:
interactive_map(hexs_brasilia_pop_access, 'Brasilia', pop_col, labels)

In [ ]:
interactive_map(hexs_florianopolis_pop_access, 'Florianopolis', pop_col, labels)